In [27]:
from glob import glob
import numpy as np
from itertools import islice

In [2]:
datas = glob("./*.data")
datas

['./N200epshm29nh36ac2nc5969mc8l25dt0.005ndump2000adump5000ens1.all.data',
 './N200epshm29nh36ac2nc5969mc8l25dt0.005ndump2000adump5000ens2.all.data']

In [ ]:
atom_dtypes = {
    'atom_id': np.int, 'mol_id': np.int, 'atom_type': np.int,
    'x': np.float, 'y': np.float, 'z': np.float, 
    'nx': np.int, 'ny': np.int, 'nz': np.int
}
velocity_dtypes = {
    'atom_id': np.int, 'vx': np.float, 'vy': np.float, 'vz': np.float, 
}
bond_dtypes = {'atom_id': np.int, 'vx': np.float, 'vy': np.float, 'vz': np.float, 
}
angle_dtypes = None

In [220]:
class lammps_data_parser:
    def __init__(self,filename):
        with open(filename, 'r') as f:
            line = f.readline() # comment 
            line = f.readline() # \n
            line = f.readline() # atoms
            words=line.split() #
            self.n_atoms = int(words[0]) 
            line=f.readline() # atom types
            words=line.split() # 
            self.t_atoms = int(words[0])
            line = f.readline() # bonds
            words = line.split() # 
            self.n_bonds=int(words[0])
            line = f.readline() # bond types
            words = line.split() # 
            self.t_bonds = int(words[0])
            line = f.readline() # angles
            words =line.split() # 
            self.n_angles=int(words[0])
            line = f.readline() # angle types
            words = line.split() # 
            self.t_angles = int(words[0])
            line = f.readline() # \n
            line = f.readline() # xmin xmax
            self.xmin, self.xmax = map(float, line.split()[:2])
            line = f.readline() # ymin ymax
            self.ymin, self.ymax = map(float, line.split()[:2])
            line = f.readline() # zmin zmax
            self.zmin, self.zmax = map(float, line.split()[:2])
            self.xlen = self.xmax - self.xmin
            self.ylen = self.ymax - self.ymin
            self.zlen = self.zmax - self.zmin
            
            self.masses = np.zeros(self.t_atoms, dtype=np.dtype('int, float'))
            line = f.readline() # \n
            line = f.readline() # Masses
            line = f.readline() # \n
            lines = islice(f, self.t_atoms)
            words = tuple(
                map(
                    lambda line: tuple(
                        int(word) if idx == 0 else float(word) \
                        for (idx, word) in enumerate(line.split()) 
                    ),
                    lines
                )
            )
            self.masses = words
            line = f.readline() # \n
            line = f.readline() # PairIJ Coeffs
            self.pair_sytle = line.split()[-1] 
            line = f.readline() # \n
            self.pair_coeffs = []
            for _ in range(int(0.5*self.t_atoms*(self.t_atoms+1))):
                line = f.readline()
                words = line.split()
                words = tuple(
                    [int(word) if idx == 0 else float(word) \
                     for idx, word in enumerate(words)])
                self.pair_coeffs.append(words)
            
            line = f.readline() # \n
            line = f.readline() # Angle Coeffs
            self.angle_sytle = line.split()[-1] 
            line = f.readline() # \n
            self.angle_coeffs = []
            for _ in range(self.t_angles):
                line = f.readline()
                words = line.split()
                words = tuple(
                    [int(word) if idx == 0 else float(word) \
                     for idx, word in enumerate(words)])
                self.angle_coeffs.append(words)
            line = f.readline() # \n
            line = f.readline() # Atoms
            line = f.readline() # \n
            lines = islice(f, self.n_atoms)
            words = list(
                map(
                    lambda line: tuple(
                        int(word) if idx not in [3, 4, 5] else float(word) \
                        for (idx, word) in enumerate(line.split())
                    ),
                    lines
                )
            )
            self.atoms = np.array(
                words,  # atom-id mol-id atom-type x y z nx ny nz
                dtype=np.dtype(
                    'int, int, int, float, float, float, int, int, int'
                )
            )
            line = f.readline() # \n
            line = f.readline() # Velocities
            line = f.readline() # \n
            lines = islice(f, self.n_atoms)
            words = list(
                map(
                    lambda line: tuple(
                        int(word) if idx == 0 else float(word) \
                        for (idx, word) in enumerate(line.split()) 
                    ),
                    lines
                )
            )
            self.velocities = np.array(
                words,  # atom-id vx vy vz
                dtype=np.dtype('int, float, float, float')
            )
            line = f.readline() # \n
            line = f.readline() # Bonds
            line = f.readline() # \n
            lines = islice(f, self.n_bonds)
            words = list(
                map(
                    lambda line: tuple(int(word) for word in line.split()),
                    lines
                )
            )
            self.bonds = np.array(
                words, # angle-id mol-id atom-1, atom-2 atom-3
                dtype=np.dtype('int, int, int, int')
            )
            line = f.readline() # \n
            line = f.readline() # Angles
            line = f.readline() # \n
            lines = islice(f, self.n_angles)
            words = list(
                map(
                    lambda line: tuple(int(word) for word in line.split()),
                    lines
                )
            )
            self.angles = np.array(
                words, # angle-id mol-id atom-1, atom-2 atom-3
                dtype=np.dtype('int, int, int, int, int')
            )

In [221]:
data_file = lammps_data_parser(datas[0])

In [233]:
def lammps_data_writer(atoms, velocities, bonds, angles, n_atoms, n_bonds, n_angles, t_atoms, t_bonds, t_angles, box_lims, fname):
    """
    Cautions:
    There is no need to define the mass types in the data file. Mass types can set in a LAMMPS input file and then written into a LAMMPS
    data file with write_data after a simulation is done.
    
    """   
    with open(fname,'w') as f:
        f.write('Lammps data files by amirhsi\n')
        f.write('\n')
        f.write('{} atoms\n'.format(n_atoms))
        f.write('{} atom types\n'.format(t_atoms))
        f.write('{} bonds\n'.format(n_bonds))
        f.write('{} bond types\n'.format(t_bonds))
        f.write('{} angles\n'.format(n_angles))
        f.write('{} angle types\n'.format(t_angles))
        f.write('\n')
        f.write('{} {} xlo xhi\n'.format(*box_lims[0]))
        f.write('{} {} ylo yhi\n'.format(*box_lims[1]))
        f.write('{} {} zlo zhi\n'.format(*box_lims[2]))
        f.write('\n')
        f.write('Atoms\n\n')
        # atom-id molecule-id atom-type x y z nx ny nz
        for atom in atoms:
            if atom
            f.write(
                '{} {} {} {} {} {} {} {} {}\n'.format(*atom)
            )
        f.write('\n')
        f.write('Velocities\n\n') 
        for vel in velocities:
            f.write(
                '{} {} {} {}\n'.format(*vel)
            )
        f.write('\n')
        f.write('Bonds\n\n')
        for bond in bonds:
            f.write(
                '{} {} {} {}\n'.format(*bond)
            )
        f.write('\n')
        f.write('Angles\n\n')
        for angle in angles:
            f.write(
                '{} {} {} {} {}\n'.format(*angle)
            )

In [235]:
lammps_data_writer(
    data_file.atoms,
    data_file.velocities,
    data_file.bonds,
    data_file.angles,
    236,
    data_file.n_bonds,
    data_file.n_angles,
    data_file.t_atoms,
    data_file.t_bonds,
    data_file.t_angles,
    [(data_file.xmin, data_file.xmax), (data_file.ymin, data_file.ymax), (data_file.zmin, data_file.zmax)], 
    'test.data')